In [ ]:
import requests
from config import ApeksAPI
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange

class LoadReport:
    def __init__(self, year, month, department_id):
        self.year = year
        self.month = month
        self.department_id = str(department_id)
        self.lessons = self.get_lessons()
        self.state_staff_history = db_filter_req('state_staff_history', 'department_id', self.department_id)

    def get_lessons(self):
        """List of lessons for selected month"""
        first_day = 1
        last_day = monthrange(self.year, self.month)[1]

        params = {
            "token": ApeksAPI.TOKEN,
            "table": 'schedule_day_schedule_lessons',
            "filter": "date between '" + date(self.year, self.month, first_day).isoformat() + "' and '" + date(self.year, self.month, last_day).isoformat() + "'"
        }
        resp = requests.get(
            ApeksAPI.URL + "/api/call/system-database/get", params=params)
        return resp.json()['data']

    def staff_list(self):
        """List of department workers which was active on selected month"""
        staff_list = []
        state_staff_positions = db_request('state_staff_positions')
        exclude_list = {'12': "инструктора произв. обучения",
                        '13': "начальник кабинета",
                        '14': "специалист по УМР",
                        '15': "зав. кабинетом",}

        for staff in self.state_staff_history:
            if staff.get('position_id') not in exclude_list:
                if staff.get('end_date') is not None:
                    if date.fromisoformat(staff.get('end_date')) > date(self.year, self.month, 1):
                        staff_list.append(staff)
                else:
                    if date.fromisoformat(staff.get('start_date')) <= date(self.year, self.month, monthrange(self.year, self.month)[1]):
                        staff_list.append(staff)

        def staff_sort(staff_id):
            """getting sorting code by position"""
            position_id = ""
            for sort_staff in staff_list:
                if sort_staff.get("staff_id") == str(staff_id):
                    position_id = sort_staff.get("position_id")
            for k in state_staff_positions:
                if k.get("id") == position_id:
                    return k.get("sort")

        def staff_name(staff_id):
            """getting staff name from DB"""
            resp = db_filter_req("state_staff", "id", staff_id)
            return f'{resp[0].get("family_name")} {resp[0].get("name")[0]}.{resp[0].get("surname")[0]}.'

        sort_dict = {}
        for i in staff_list:
            sort_dict[i['staff_id']] = int(staff_sort(i['staff_id']))
        a = sorted(sort_dict.items(), key=lambda x: x[1], reverse=True)
        prepod_dict = {}
        for i in range(len(a)):
            prepod_dict[a[i][0]] = staff_name(a[i][0])
        return prepod_dict